In [29]:
import os, sys, h5py, hdf5plugin, glob, numpy as np
from typing import List

try:
    import cupy as cp; xp, GPU = cp, True
    print("CuPy found – running on GPU")
except ImportError:
    import numpy as xp; GPU = False
    print("CuPy not found – falling back to NumPy (CPU)")

import napari
from magicgui import magicgui
from magicgui.widgets import Label, PushButton, LineEdit
from napari.qt.threading import thread_worker
from qtpy.QtCore    import Qt, QTimer
from qtpy.QtWidgets import (QApplication, QWidget, QListWidget,
                            QListWidgetItem, QSpinBox, QLabel,
                            QHBoxLayout, QVBoxLayout, QFileDialog)

def launch_viewer(root_dir, motors_file):
    root_dir = root_dir.rstrip('/') + '/'
    
    # Fix: Use proper glob pattern to find actual files
    scan_pattern = root_dir + "scan*/DEPow_0.000000_rockinglayer_10x_*_converted.h5"
    scan_files = sorted(glob.glob(scan_pattern))
    
    if not scan_files:
        raise RuntimeError(f"No scan files found matching pattern: {scan_pattern}")
    
    print(f"Found {len(scan_files)} scan files:")
    for f in scan_files[:3]:  # Print first 3 as preview
        print(f"  {f}")
    
    # Open the FIRST actual file found (not the pattern)
    with h5py.File(scan_files[0]) as f0:
        SCAN_COUNT, H, W = f0['scan_0/image/data'].shape

    with h5py.File(motors_file) as f:
        phi_vals = np.round(f['scan_0/instrument/positioners/diffry'][:], 4)
        chi_vals = np.round(f['scan_0/instrument/positioners/chi'][:], 4)

    PHI_COUNT = np.unique(phi_vals).size
    CHI_COUNT = np.unique(chi_vals).size

    def _frame(path: str, χ: int, φ: int) -> np.ndarray:
        χ = np.clip(χ, 0, CHI_COUNT-1)
        φ = np.clip(φ, 0, PHI_COUNT-1)
        g = φ*CHI_COUNT + χ
        with h5py.File(path) as f:
            return f['scan_0/image/data'][g]

    def _mean(path: str, χ0: int, φ0: int, χr: int, φr: int) -> np.ndarray:
        acc, cnt = None, 0
        for dχ in range(-χr, χr+1):
            for dφ in range(-φr, φr+1):
                f = _frame(path, χ0+dχ, φ0+dφ)
                acc = f if acc is None else acc + f
                cnt += 1
        return acc / cnt

    current_chi = 0
    current_phi = 42
    current_chi_rad = 0
    current_phi_rad = 0
    local_chi_off = np.zeros(len(scan_files), dtype=int)
    local_phi_off  = np.zeros(len(scan_files), dtype=int)
    slice_visible = np.ones(len(scan_files), dtype=bool)

    @thread_worker
    def _volume_worker(χ, φ, χr, φr, χoff, φoff, vis):
        vol = []
        for i, path in enumerate(scan_files):
            if not vis[i]:
                vol.append(np.zeros((H, W), dtype=np.float32)); continue
            eff_χ = χ + int(χoff[i])
            eff_φ = φ + int(φoff[i])
            vol.append(_mean(path, eff_χ, eff_φ, χr, φr))
        return np.log10(np.stack(vol).astype(np.float32))

    first_vol = _volume_worker.__wrapped__(
        current_chi, current_phi, current_chi_rad, current_phi_rad,
        local_chi_off, local_phi_off, slice_visible
    )
    vmin, vmax = float(first_vol.min()), float(first_vol.max())
    dz, dx, dy = 5.0, 0.15, 0.15

    viewer = napari.Viewer(title="DFXM volume (χ, φ)")
    vol_layer = viewer.add_image(
        first_vol, name='intensity', contrast_limits=(vmin, vmax),
        colormap='viridis', rendering='mip',
        iso_threshold=vmin + 0.25*(vmax - vmin), attenuation=0.01,
        scale=(dz, dy, dx)
    )
    chi_phi_label = Label()

    def _status():
        g = np.clip(current_phi,0,PHI_COUNT-1)*CHI_COUNT + np.clip(current_chi,0,CHI_COUNT-1)
        chi_phi_label.value = f"χ = {chi_vals[g]:.3f}°,   φ = {phi_vals[g]:.3f}°"
        viewer.status = f"χ‑idx {current_chi}, φ‑idx {current_phi} · "+chi_phi_label.value

    def _refresh():
        worker = _volume_worker(
            current_chi, current_phi, current_chi_rad, current_phi_rad,
            local_chi_off.copy(), local_phi_off.copy(), slice_visible.copy()
        )
        worker.returned.connect(lambda d: setattr(vol_layer, "data", d))
        worker.returned.connect(lambda _: _status())
        worker.start()

    @magicgui(χ_idx={'widget_type':'Slider','min':0,'max':CHI_COUNT-1,
                     'orientation':'horizontal','tracking':False},
              auto_call=True)
    def χ_slider(χ_idx: int = 0):
        nonlocal current_chi; current_chi = int(χ_idx); _refresh()

    @magicgui(φ_idx={'widget_type':'Slider','min':0,'max':PHI_COUNT-1,
                     'orientation':'horizontal','tracking':False},
              auto_call=True)
    def φ_slider(φ_idx: int = 0):
        nonlocal current_phi; current_phi = int(φ_idx); _refresh()

    @magicgui(χ_rad={'widget_type':'SpinBox','min':0,'max':CHI_COUNT//2},
              auto_call=True, layout='horizontal')
    def χ_comb(χ_rad: int = 0):
        nonlocal current_chi_rad; current_chi_rad = int(χ_rad); _refresh()

    @magicgui(φ_rad={'widget_type':'SpinBox','min':0,'max':PHI_COUNT//2},
              auto_call=True, layout='horizontal')
    def φ_comb(φ_rad: int = 0):
        nonlocal current_phi_rad; current_phi_rad = int(φ_rad); _refresh()

    gbox = QWidget(); g_lay = QVBoxLayout(gbox)
    for w in (χ_slider.native, φ_slider.native,
              χ_comb.native, φ_comb.native, chi_phi_label.native):
        g_lay.addWidget(w)

    class SliceCtrl(QWidget):
        def __init__(self):
            super().__init__()
            lay = QVBoxLayout(self)
            self.list = QListWidget()
            self.list.setSelectionMode(QListWidget.MultiSelection)
            for i in range(len(scan_files)):
                item = QListWidgetItem(f"z {i}")
                item.setFlags(item.flags() | Qt.ItemIsUserCheckable)
                item.setCheckState(Qt.Checked)
                self.list.addItem(item)
            lay.addWidget(self.list)

            rowχ = QHBoxLayout()
            rowχ.addWidget(QLabel("Δχ")); self.chi_sp = QSpinBox()
            self.chi_sp.setRange(-CHI_COUNT, CHI_COUNT); rowχ.addWidget(self.chi_sp)
            lay.addLayout(rowχ)

            rowφ = QHBoxLayout()
            rowφ.addWidget(QLabel("Δφ")); self.phi_sp = QSpinBox()
            self.phi_sp.setRange(-PHI_COUNT, PHI_COUNT); rowφ.addWidget(self.phi_sp)
            lay.addLayout(rowφ)

            self.list.itemSelectionChanged.connect(self._sync)
            self.list.itemChanged.connect(self._vis_change)
            self.chi_sp.valueChanged.connect(self._apply)
            self.phi_sp.valueChanged.connect(self._apply)

        def _rows(self): return [i.row() for i in self.list.selectedIndexes()]

        def _sync(self):
            if not self._rows(): return
            r = self._rows()[0]
            self.chi_sp.blockSignals(True); self.phi_sp.blockSignals(True)
            self.chi_sp.setValue(int(local_chi_off[r]))
            self.phi_sp.setValue(int(local_phi_off[r]))
            self.chi_sp.blockSignals(False); self.phi_sp.blockSignals(False)

        def _apply(self):
            for r in self._rows():
                local_chi_off[r] = self.chi_sp.value()
                local_phi_off[r]  = self.phi_sp.value()
            _refresh()

        def _vis_change(self, item):
            row = self.list.row(item)
            slice_visible[row] = (item.checkState()==Qt.Checked)
            _refresh()

    local_box = SliceCtrl()

    class FileBox(QWidget):
        def __init__(self):
            super().__init__()
            lay = QVBoxLayout(self)
            self.root_edit   = LineEdit(value=root_dir, label="Root folder")
            self.motor_edit  = LineEdit(value=motors_file, label="Motors .h5")
            browse_btn = PushButton(text="Browse folder…")
            motor_btn  = PushButton(text="Browse file…")
            apply_btn  = PushButton(text="Apply & Reload")

            row1 = QHBoxLayout(); row1.addWidget(self.root_edit.native)
            row1.addWidget(browse_btn.native); lay.addLayout(row1)
            row2 = QHBoxLayout(); row2.addWidget(self.motor_edit.native)
            row2.addWidget(motor_btn.native); lay.addLayout(row2)
            lay.addWidget(apply_btn.native)

            browse_btn.clicked.connect(self._pick_folder)
            motor_btn .clicked.connect(self._pick_file)
            apply_btn .clicked.connect(self._apply)

        def _pick_folder(self):
            d = QFileDialog.getExistingDirectory(self, "Select scan folder", root)
            if d: self.root_edit.value = d if d.endswith(os.sep) else d+os.sep

        def _pick_file(self):
            f, _ = QFileDialog.getOpenFileName(self, "Select motors HDF5",
                                               os.path.dirname(motors_file),
                                               "HDF5 files (*.h5)")
            if f: self.motor_edit.value = f

        def _apply(self):
            new_root  = self.root_edit.value
            new_motor = self.motor_edit.value
            viewer.close()
            QTimer.singleShot(0, lambda: launch_viewer(new_root, new_motor))

    file_box = FileBox()

    viewer.window.add_dock_widget(local_box,name="LOCAL slice tools", area="right")
    viewer.window.add_dock_widget(gbox,     name="GLOBAL controls", area="right")
    viewer.window.add_dock_widget(file_box, name="FILE handling", area="right")

    app = QApplication.instance()
    app.setStyleSheet(app.styleSheet()+"QLabeledSlider > QAbstractSpinBox {padding:0px 4px;}")

    _status()


    from napari_animation import Animation

    animation = Animation(viewer)

    viewer.dims.ndisplay = 3
    viewer.camera.angles = (0.0, 0.0, 90.0)
    animation.capture_keyframe()
    viewer.camera.zoom = 2.4
    animation.capture_keyframe()
    viewer.camera.angles = (-7.0, 15.7, 62.4)
    animation.capture_keyframe(steps=60)
    viewer.camera.angles = (2.0, -24.4, -36.7)
    animation.capture_keyframe(steps=60)
    viewer.reset_view()
    viewer.camera.angles = (0.0, 0.0, 90.0)
    animation.capture_keyframe()
    animation.animate('demo.mov', canvas_only=True)

    viewer.show()

if __name__ == "__main__":
    init_root   = "/Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/"
    init_motors = init_root + "DEPow_0.000000_rockinglayer_10x_00_converted.h5"
    launch_viewer(init_root, init_motors)
    napari.run()



CuPy not found – falling back to NumPy (CPU)
Found 21 scan files:
  /Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/scan0000/DEPow_0.000000_rockinglayer_10x_00_converted.h5
  /Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/scan0001/DEPow_0.000000_rockinglayer_10x_01_converted.h5
  /Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/scan0002/DEPow_0.000000_rockinglayer_10x_02_converted.h5
Rendering frames...


100%|██████████| 151/151 [00:11<00:00, 13.39it/s]
2025-12-29 10:15:26.567 Python[18791:27751491] not in fullscreen state


In [17]:
pip install napari-animation


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install imageio-ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.0 MB/s  0:00:00 eta 0:00:01

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os, sys, h5py, hdf5plugin, glob, numpy as np
from typing import List

try:
    import cupy as cp; xp, GPU = cp, True
    print("CuPy found – running on GPU")
except ImportError:
    import numpy as xp; GPU = False
    print("CuPy not found – falling back to NumPy (CPU)")

import napari
from magicgui import magicgui
from magicgui.widgets import Label, PushButton, LineEdit
from napari.qt.threading import thread_worker
from qtpy.QtCore    import Qt, QTimer
from qtpy.QtWidgets import (QApplication, QWidget, QListWidget,
                            QListWidgetItem, QSpinBox, QLabel,
                            QHBoxLayout, QVBoxLayout, QFileDialog)
from vispy import scene
from vispy.io import write_png
import imageio

def launch_viewer(root_dir, motors_file):
    root_dir = root_dir.rstrip('/') + '/'
    
    # Create output folder if it doesn't exist
    output_dir = os.path.join(root_dir, "output")
    os.makedirs(output_dir, exist_ok=True)
    
    # Fix: Use proper glob pattern to find actual files
    scan_pattern = root_dir + "scan*/DEPow_0.000000_rockinglayer_10x_*_converted.h5"
    scan_files = sorted(glob.glob(scan_pattern))
    
    if not scan_files:
        raise RuntimeError(f"No scan files found matching pattern: {scan_pattern}")
    
    print(f"Found {len(scan_files)} scan files:")
    for f in scan_files[:3]:  # Print first 3 as preview
        print(f"  {f}")
    
    # Open the FIRST actual file found (not the pattern)
    with h5py.File(scan_files[0]) as f0:
        SCAN_COUNT, H, W = f0['scan_0/image/data'].shape

    with h5py.File(motors_file) as f:
        phi_vals = np.round(f['scan_0/instrument/positioners/diffry'][:], 4)
        chi_vals = np.round(f['scan_0/instrument/positioners/chi'][:], 4)

    PHI_COUNT = np.unique(phi_vals).size
    CHI_COUNT = np.unique(chi_vals).size

    def _frame(path: str, χ: int, φ: int) -> np.ndarray:
        χ = np.clip(χ, 0, CHI_COUNT-1)
        φ = np.clip(φ, 0, PHI_COUNT-1)
        g = φ*CHI_COUNT + χ
        with h5py.File(path) as f:
            return f['scan_0/image/data'][g]

    def _mean(path: str, χ0: int, φ0: int, χr: int, φr: int) -> np.ndarray:
        acc, cnt = None, 0
        for dχ in range(-χr, χr+1):
            for dφ in range(-φr, φr+1):
                f = _frame(path, χ0+dχ, φ0+dφ)
                acc = f if acc is None else acc + f
                cnt += 1
        return acc / cnt

    current_chi = 0
    current_phi = 0
    current_chi_rad = 0
    current_phi_rad = 0
    local_chi_off = np.zeros(len(scan_files), dtype=int)
    local_phi_off  = np.zeros(len(scan_files), dtype=int)
    slice_visible = np.ones(len(scan_files), dtype=bool)

    @thread_worker
    def _volume_worker(χ, φ, χr, φr, χoff, φoff, vis):
        vol = []
        for i, path in enumerate(scan_files):
            if not vis[i]:
                vol.append(np.zeros((H, W), dtype=np.float32)); continue
            eff_χ = χ + int(χoff[i])
            eff_φ = φ + int(φoff[i])
            vol.append(_mean(path, eff_χ, eff_φ, χr, φr))
        return np.log10(np.stack(vol).astype(np.float32))

    first_vol = _volume_worker.__wrapped__(
        current_chi, current_phi, current_chi_rad, current_phi_rad,
        local_chi_off, local_phi_off, slice_visible
    )
    vmin, vmax = float(first_vol.min()), float(first_vol.max())
    dz, dx, dy = 5.0, 0.15, 0.15

    viewer = napari.Viewer(title="DFXM volume (χ, φ)")
    vol_layer = viewer.add_image(
        first_vol, name='intensity', contrast_limits=(vmin, vmax),
        colormap='viridis', rendering='iso',
        iso_threshold=vmin + 0.25*(vmax - vmin), attenuation=0.01,
        scale=(dz, dy, dx)
    )
    chi_phi_label = Label()

    def _status():
        g = np.clip(current_phi,0,PHI_COUNT-1)*CHI_COUNT + np.clip(current_chi,0,CHI_COUNT-1)
        chi_phi_label.value = f"χ = {chi_vals[g]:.3f}°,   φ = {phi_vals[g]:.3f}°"
        viewer.status = f"χ‑idx {current_chi}, φ‑idx {current_phi} · "+chi_phi_label.value

    def _refresh():
        worker = _volume_worker(
            current_chi, current_phi, current_chi_rad, current_phi_rad,
            local_chi_off.copy(), local_phi_off.copy(), slice_visible.copy()
        )
        worker.returned.connect(lambda d: setattr(vol_layer, "data", d))
        worker.returned.connect(lambda _: _status())
        worker.start()

    @magicgui(χ_idx={'widget_type':'Slider','min':0,'max':CHI_COUNT-1,
                     'orientation':'horizontal','tracking':False},
              auto_call=True)
    def χ_slider(χ_idx: int = 0):
        nonlocal current_chi; current_chi = int(χ_idx); _refresh()

    @magicgui(φ_idx={'widget_type':'Slider','min':0,'max':PHI_COUNT-1,
                     'orientation':'horizontal','tracking':False},
              auto_call=True)
    def φ_slider(φ_idx: int = 0):
        nonlocal current_phi; current_phi = int(φ_idx); _refresh()

    @magicgui(χ_rad={'widget_type':'SpinBox','min':0,'max':CHI_COUNT//2},
              auto_call=True, layout='horizontal')
    def χ_comb(χ_rad: int = 0):
        nonlocal current_chi_rad; current_chi_rad = int(χ_rad); _refresh()

    @magicgui(φ_rad={'widget_type':'SpinBox','min':0,'max':PHI_COUNT//2},
              auto_call=True, layout='horizontal')
    def φ_comb(φ_rad: int = 0):
        nonlocal current_phi_rad; current_phi_rad = int(φ_rad); _refresh()

    gbox = QWidget(); g_lay = QVBoxLayout(gbox)
    for w in (χ_slider.native, φ_slider.native,
              χ_comb.native, φ_comb.native, chi_phi_label.native):
        g_lay.addWidget(w)

    class SliceCtrl(QWidget):
        def __init__(self):
            super().__init__()
            lay = QVBoxLayout(self)
            self.list = QListWidget()
            self.list.setSelectionMode(QListWidget.MultiSelection)
            for i in range(len(scan_files)):
                item = QListWidgetItem(f"z {i}")
                item.setFlags(item.flags() | Qt.ItemIsUserCheckable)
                item.setCheckState(Qt.Checked)
                self.list.addItem(item)
            lay.addWidget(self.list)

            rowχ = QHBoxLayout()
            rowχ.addWidget(QLabel("Δχ")); self.chi_sp = QSpinBox()
            self.chi_sp.setRange(-CHI_COUNT, CHI_COUNT); rowχ.addWidget(self.chi_sp)
            lay.addLayout(rowχ)

            rowφ = QHBoxLayout()
            rowφ.addWidget(QLabel("Δφ")); self.phi_sp = QSpinBox()
            self.phi_sp.setRange(-PHI_COUNT, PHI_COUNT); rowφ.addWidget(self.phi_sp)
            lay.addLayout(rowφ)

            self.list.itemSelectionChanged.connect(self._sync)
            self.list.itemChanged.connect(self._vis_change)
            self.chi_sp.valueChanged.connect(self._apply)
            self.phi_sp.valueChanged.connect(self._apply)

        def _rows(self): return [i.row() for i in self.list.selectedIndexes()]

        def _sync(self):
            if not self._rows(): return
            r = self._rows()[0]
            self.chi_sp.blockSignals(True); self.phi_sp.blockSignals(True)
            self.chi_sp.setValue(int(local_chi_off[r]))
            self.phi_sp.setValue(int(local_phi_off[r]))
            self.chi_sp.blockSignals(False); self.phi_sp.blockSignals(False)

        def _apply(self):
            for r in self._rows():
                local_chi_off[r] = self.chi_sp.value()
                local_phi_off[r]  = self.phi_sp.value()
            _refresh()

        def _vis_change(self, item):
            row = self.list.row(item)
            slice_visible[row] = (item.checkState()==Qt.Checked)
            _refresh()

    local_box = SliceCtrl()

    class MovieBox(QWidget):
        """Widget for creating rotating movies"""
        def __init__(self):
            super().__init__()
            lay = QVBoxLayout(self)
            
            lay.addWidget(QLabel("<b>Movie Creation</b>"))
            
            # Number of frames
            frames_row = QHBoxLayout()
            frames_row.addWidget(QLabel("Frames:"))
            self.frames_sp = QSpinBox()
            self.frames_sp.setRange(10, 500)
            self.frames_sp.setValue(120)
            frames_row.addWidget(self.frames_sp)
            lay.addLayout(frames_row)
            
            # FPS
            fps_row = QHBoxLayout()
            fps_row.addWidget(QLabel("FPS:"))
            self.fps_sp = QSpinBox()
            self.fps_sp.setRange(5, 60)
            self.fps_sp.setValue(30)
            fps_row.addWidget(self.fps_sp)
            lay.addLayout(fps_row)
            
            # Image size
            size_row = QHBoxLayout()
            size_row.addWidget(QLabel("Width:"))
            self.width_sp = QSpinBox()
            self.width_sp.setRange(400, 3840)
            self.width_sp.setValue(1920)
            self.width_sp.setSingleStep(100)
            size_row.addWidget(self.width_sp)
            
            size_row.addWidget(QLabel("Height:"))
            self.height_sp = QSpinBox()
            self.height_sp.setRange(400, 2160)
            self.height_sp.setValue(1080)
            self.height_sp.setSingleStep(100)
            size_row.addWidget(self.height_sp)
            lay.addLayout(size_row)
            
            # Create buttons for different rotations
            self.z_btn = PushButton(text="Create Z-axis rotation")
            self.y_btn = PushButton(text="Create Y-axis rotation")
            self.zy_btn = PushButton(text="Create Z+Y rotation")
            
            lay.addWidget(self.z_btn.native)
            lay.addWidget(self.y_btn.native)
            lay.addWidget(self.zy_btn.native)
            
            self.status_label = QLabel("")
            lay.addWidget(self.status_label)
            
            self.z_btn.clicked.connect(lambda: self._create_movie('z'))
            self.y_btn.clicked.connect(lambda: self._create_movie('y'))
            self.zy_btn.clicked.connect(lambda: self._create_movie('zy'))
        
        def _create_movie(self, rotation_type):
            """Create rotating movie"""
            n_frames = self.frames_sp.value()
            fps = self.fps_sp.value()
            width = self.width_sp.value()
            height = self.height_sp.value()
            
            self.status_label.setText("Rendering movie...")
            QApplication.processEvents()
            
            # Get current camera state
            camera = viewer.camera
            initial_angles = camera.angles
            initial_center = camera.center
            
            frames = []
            
            for i in range(n_frames):
                if rotation_type == 'z':
                    # Rotate around Z-axis
                    angle_z = 360 * i / n_frames
                    camera.angles = (initial_angles[0], angle_z, initial_angles[2])
                    
                elif rotation_type == 'y':
                    # Rotate around Y-axis
                    angle_y = 360 * i / n_frames
                    camera.angles = (angle_y, initial_angles[1], initial_angles[2])
                    
                elif rotation_type == 'zy':
                    # Rotate around both Z and Y axes
                    angle_z = 360 * i / n_frames
                    angle_y = 180 * np.sin(2 * np.pi * i / n_frames)
                    camera.angles = (angle_y, angle_z, initial_angles[2])
                
                # Force update
                viewer.window._qt_window.update()
                QApplication.processEvents()
                
                # Capture frame
                screenshot = viewer.screenshot(canvas_only=True, size=(width, height))
                frames.append(screenshot)
                
                # Update status
                self.status_label.setText(f"Rendering: {i+1}/{n_frames}")
                QApplication.processEvents()
            
            # Reset camera
            camera.angles = initial_angles
            
            # Save movie
            timestamp = np.datetime64('now').astype(str).replace(':', '-').replace('.', '-')
            output_path = os.path.join(output_dir, f"rotation_{rotation_type}_{timestamp}.mp4")
            
            self.status_label.setText("Encoding video...")
            QApplication.processEvents()
            
            try:
                imageio.mimsave(output_path, frames, fps=fps, codec='libx264', quality=8)
                self.status_label.setText(f"✓ Saved: {os.path.basename(output_path)}")
                print(f"Movie saved to: {output_path}")
            except Exception as e:
                self.status_label.setText(f"✗ Error: {str(e)}")
                print(f"Error saving movie: {e}")

    movie_box = MovieBox()

    class FileBox(QWidget):
        def __init__(self):
            super().__init__()
            lay = QVBoxLayout(self)
            self.root_edit   = LineEdit(value=root_dir, label="Root folder")
            self.motor_edit  = LineEdit(value=motors_file, label="Motors .h5")
            browse_btn = PushButton(text="Browse folder…")
            motor_btn  = PushButton(text="Browse file…")
            apply_btn  = PushButton(text="Apply & Reload")

            row1 = QHBoxLayout(); row1.addWidget(self.root_edit.native)
            row1.addWidget(browse_btn.native); lay.addLayout(row1)
            row2 = QHBoxLayout(); row2.addWidget(self.motor_edit.native)
            row2.addWidget(motor_btn.native); lay.addLayout(row2)
            lay.addWidget(apply_btn.native)

            browse_btn.clicked.connect(self._pick_folder)
            motor_btn .clicked.connect(self._pick_file)
            apply_btn .clicked.connect(self._apply)

        def _pick_folder(self):
            d = QFileDialog.getExistingDirectory(self, "Select scan folder", root_dir)
            if d: self.root_edit.value = d if d.endswith(os.sep) else d+os.sep

        def _pick_file(self):
            f, _ = QFileDialog.getOpenFileName(self, "Select motors HDF5",
                                               os.path.dirname(motors_file),
                                               "HDF5 files (*.h5)")
            if f: self.motor_edit.value = f

        def _apply(self):
            new_root  = self.root_edit.value
            new_motor = self.motor_edit.value
            viewer.close()
            QTimer.singleShot(0, lambda: launch_viewer(new_root, new_motor))

    file_box = FileBox()

    viewer.window.add_dock_widget(local_box,name="LOCAL slice tools", area="right")
    viewer.window.add_dock_widget(gbox,     name="GLOBAL controls", area="right")
    viewer.window.add_dock_widget(movie_box, name="MOVIE creation", area="right")
    viewer.window.add_dock_widget(file_box, name="FILE handling", area="right")

    app = QApplication.instance()
    app.setStyleSheet(app.styleSheet()+"QLabeledSlider > QAbstractSpinBox {padding:0px 4px;}")

    _status()
    viewer.show()

if __name__ == "__main__":
    init_root   = "/Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/"
    init_motors = init_root + "DEPow_0.000000_rockinglayer_10x_00_converted.h5"
    launch_viewer(init_root, init_motors)
    napari.run()

CuPy not found – falling back to NumPy (CPU)
Found 21 scan files:
  /Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/scan0000/DEPow_0.000000_rockinglayer_10x_00_converted.h5
  /Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/scan0001/DEPow_0.000000_rockinglayer_10x_01_converted.h5
  /Users/edemdoehonu/Library/CloudStorage/GoogleDrive-edemdh@stanford.edu/My Drive/Morning_after_H5files/scan0002/DEPow_0.000000_rockinglayer_10x_02_converted.h5


  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/edemdoehonu/Library/Python/3.13/lib/python/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/edemdoehonu/Library/Python/3.13/lib/python/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/edemdoehonu/Library/Python/3.13/lib/python/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/Users/edemdoehonu/Library/Python/3.13/lib/python/site-packages/tornado/platform/asyncio.py", line 211, in start
    self.asyncio_loop.run_forever()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/base_events.py", line 683, in run_forever
    self._run_once()
  File "/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/asyncio/base_events.py", line 2050, in _run_once
    handle._run()
  File "/Lib

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
File /Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/napari/_qt/threads/status_checker.py:126, in StatusChecker.calculate_status(self=<napari._qt.threads.status_checker.StatusChecker object>)
    122     return
    124 try:
    125     # Calculate the status change from cursor's movement
--> 126     res = viewer._calc_status_from_cursor()
        viewer = Viewer(camera=Camera(center=(np.float64(50.0), np.float64(161.925), np.float64(191.925)), zoom=np.float64(1.714453125), angles=(np.float64(-78.34826665400831), np.float64(-51.55700347134654), np.float64(-1.7032856900128794)), perspective=0.0, mouse_pan=True, mouse_zoom=True, orientation=(<DepthAxisOrientation.TOWARDS: 'towards'>, <VerticalAxisOrientation.DOWN: 'down'>, <HorizontalAxisOrientation.RIGHT: 'right'>)), cursor=Cursor(position=(np.float64(-3.71